<a href="https://colab.research.google.com/github/dfukagaw28/ColabNotebooks/blob/main/%E6%96%87%E5%AD%97%E3%82%B3%E3%83%BC%E3%83%89%E3%81%AB%E3%81%A4%E3%81%84%E3%81%A6_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 文字コードについて (1)

In [1]:
!iconv --version

iconv (Ubuntu GLIBC 2.35-0ubuntu3.4) 2.35
Copyright (C) 2022 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
Written by Ulrich Drepper.


## いわゆる「シフトJIS」の取り扱い

In [2]:
# UTF-8 の文字コード（3バイト）
!echo -n あ | xxd -p

e38182


In [3]:
# Shift_JIS の文字コード（2バイト）
!echo -n あ | iconv -t shift_jis | xxd -p

82a0


## いわゆる「機種依存文字」を「シフトJIS」で扱う

*   機種依存文字はシフト JIS で扱えない

In [4]:
# 丸付き数字を UTF-8 で
!echo -n ① | xxd -p

e291a0


In [5]:
# 丸付き数字は iconv の shift_jis では扱えない
!echo -n ① | iconv -t shift_jis | xxd -p

iconv: illegal input sequence at position 0


In [6]:
# 丸付き数字は「NEC特殊文字」のひとつであり， iconv の cp932 で扱える
!echo -n ① | iconv -t cp932 | xxd -p

8740


## いわゆる「全角ダッシュ」問題

*   [全角ダッシュのマッピング問題](https://ja.wikipedia.org/wiki/%E3%83%80%E3%83%83%E3%82%B7%E3%83%A5_(%E8%A8%98%E5%8F%B7)#%E5%85%A8%E8%A7%92%E3%83%80%E3%83%83%E3%82%B7%E3%83%A5%E3%81%AE%E3%83%9E%E3%83%83%E3%83%94%E3%83%B3%E3%82%B0%E5%95%8F%E9%A1%8C)

*   U+2014 (EM DASH) も U+2015 (HORIZONTAL BAR) も， iconv で cp932 に変換すると `81 5c` になる

In [7]:
# EM DASH
!echo -e '\u2014'
!echo -en '\u2014' | xxd -p
!echo -n 'shift_jis: ' && echo -en '\u2014' | iconv -t shift_jis | xxd -p
!echo -n 'cp932:     ' && echo -en '\u2014' | iconv -t cp932 | xxd -p

—
e28094
shift_jis: iconv: illegal input sequence at position 0
cp932:     815c


In [8]:
# HORIZONTAL BAR
!echo -e '\u2015'
!echo -en '\u2015' | xxd -p
!echo -n 'shift_jis: ' && echo -en '\u2015' | iconv -t shift_jis | xxd -p
!echo -n 'cp932:     ' && echo -en '\u2015' | iconv -t cp932 | xxd -p

―
e28095
shift_jis: 815c
cp932:     815c


*   逆に， `81 5c` を UTF-8 に変換するとどうなるのか
    *   定義上は EM DASH つまり U+2014 になるはず
    *   ところが shift_jis でも cp932 でも U+2015 に変換される
    *   JIS X 0213:2000

In [9]:
!echo -n 'shift_jis: ' && echo -en '\x81\x5c' | iconv -f shift_jis | xxd -p
!echo -n 'cp932:     ' && echo -en '\x81\x5c' | iconv -f cp932 | xxd -p

shift_jis: e28095
cp932:     e28095


*   iconv によって EM DASH が U+2014 に変換されるようにするには，文字コードとして SHIFT_JISX0213 を指定すればよい

In [10]:
!echo -n 'shift_jisx0213: ' && echo -en '\x81\x5c' | iconv -f shift_jisx0213 | xxd -p

shift_jisx0213: e28094


## 「はしごだか」を扱う

*   cp932 ならうまくいくが，shift_jisx0213 では変換できない

In [11]:
!echo -n 髙 | xxd -p
!echo -n 髙 | iconv -t shift_jis | xxd -p
!echo -n 髙 | iconv -t cp932 | xxd -p
!echo -n 髙 | iconv -t shift_jisx0213 | xxd -p

e9ab99
iconv: illegal input sequence at position 0
fbfc
iconv: illegal input sequence at position 0


## その他の候補

In [12]:
!iconv -l | grep -i shift
!iconv -l | grep -i sjis
!iconv -l | grep -i 932

CSSHIFTJIS//
SHIFT-JIS//
SHIFTJISX0213//
SHIFT_JIS//
SHIFT_JISX0213//
SJIS-OPEN//
SJIS-WIN//
SJIS//
CP932//
CSIBM932//
IBM-932//
IBM932//
MS932//


In [13]:
import subprocess
import pandas as pd

encodings = [
    'UTF8',
    'SJIS',
    #'SHIFT-JIS',       # SJIS と同じ（？）
    #'SHIFT_JIS',       # SJIS と同じ（？）
    #'CSSHIFTJIS',      # SJIS と同じ（？）
    'IBM932',
    #'IBM-932',         # IBM932 と同じ（？）
    #'CSIBM932',        # IBM932 と同じ（？）
    'CP932',
    #'MS932',           # CP932 と同じ（？）
    #'SJIS-OPEN',       # CP932 と同じ（？）
    #'SJIS-WIN',        # CP932 と同じ（？）
    'SHIFTJISX0213',
    #'SHIFT_JISX0213',  # SHIFTJISX0213 と同じ（？）
]

texts = ['—', '―', 'Ⅰ', 'ⅰ', '①', '㈱', '髙', '﨑']

df = pd.DataFrame(index=texts, columns=encodings)

for text in texts:
  for encoding in encodings:
    try:
      command = ['iconv', '-t', encoding]
      result = subprocess.run(command, input=text.encode('utf8'), capture_output=True)
      df.loc[text, encoding] = result.stdout.hex()
      if result.stdout.hex() != '':
        command = ['iconv', '-f', encoding]
        result = subprocess.run(command, input=result.stdout, capture_output=True)
        rev = 'o' if result.stdout == text.encode('utf8') else 'x'
        df.loc[text, encoding] += ' ' + rev
    except:
      pass

display(df)

,UTF8,SJIS,IBM932,CP932,SHIFTJISX0213
—,e28094 o,,815c o,815c x,815c o
―,e28095 o,815c o,815c x,815c o,
Ⅰ,e285a0 o,,fa4a o,8754 o,8754 o
ⅰ,e285b0 o,,fa40 o,fa40 o,86b3 o
①,e291a0 o,,,8740 o,8740 o
㈱,e388b1 o,,fa58 o,878a o,878a o
髙,e9ab99 o,,fbfc o,fbfc o,
﨑,efa891 o,,fab1 o,fab1 o,9892 o
